## California Housing Datensatz 

## Aufgabe 1: Multiple Regressionsanalyse California Housing

### Datensatz und Tools laden

In [8]:
from sklearn.datasets import fetch_california_housing
df = fetch_california_housing(as_frame=True).frame #gibt einen panda datenframe zurück
print(df)

       MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0      8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1      8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2      7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3      5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4      3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   
...       ...       ...       ...        ...         ...       ...       ...   
20635  1.5603      25.0  5.045455   1.133333       845.0  2.560606     39.48   
20636  2.5568      18.0  6.114035   1.315789       356.0  3.122807     39.49   
20637  1.7000      17.0  5.205543   1.120092      1007.0  2.325635     39.43   
20638  1.8672      18.0  5.329513   1.171920       741.0  2.123209     39.43   
20639  2.3886      16.0  5.254717   1.162264      1387.0  2.616981     39.37   

       Longitude  MedHouseVal  
0      

3 sinnvolle Prädiktoren wären: AveOccup, Averooms, MedInc

Begründung:
- Begründung für die Wahl: Paare können sich teurere Wohnungen leisten als singles -> Einfluss auf Häuserpreis
- Begründung für die Wahl: Mehr Räume lassen auf größere Häuser schließen, welche teurer sind -> Einfluss auf Häuserpreis
- Begründung für die Wahl: Menschen mit geringerem Einkommen können sich keine teuren Häuser leisten -> Einfluss auf Häuserpreis

### Modell erstellen

In [9]:
import statsmodels.api as sm

y = df["MedHouseVal"]
features = ["AveOccup", "AveRooms", "MedInc"]
x = df[features]

# intercept hinzufügen
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            MedHouseVal   R-squared:                       0.481
Model:                            OLS   Adj. R-squared:                  0.481
Method:                 Least Squares   F-statistic:                     6370.
Date:                Thu, 27 Nov 2025   Prob (F-statistic):               0.00
Time:                        13:52:03   Log-Likelihood:                -25477.
No. Observations:               20640   AIC:                         5.096e+04
Df Residuals:                   20636   BIC:                         5.099e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6069      0.016     37.444      0.0

### Ergebnisse interpretieren

- P Wert ist überall kleiner als 1%
- F-Statistic Größer als 1 zeigt, dass das Modell valide ist
- R-squared: Wir können 48% der Gesamtvarianz erklären
- Koeffizient von AverRooms ist negativ: Luxusarpartements mit wenigen Räumen sind vielleicht teurerer als Häuser im Hinterland
- Wenn Adj. R-squared genause groß ist wie R-squared bedeutet das, dass das modell nicht durch zu viel prädiktoren aufgebläht wurde

## Aufgabe 2: Vorhersage California Housing

### Train-Test-Aufteilung

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=999) #20% Testdaten
reg = LinearRegression().fit(x_train, y_train)

### Vorhersage und Modellbewertung

In [11]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_pred = reg.predict(x_test)

# Fehlermetriken berechnen
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)

print("Mean squared error: ", mse)
print("Root mean squared error: ", rmse)

Mean squared error:  0.7124969540218331
Root mean squared error:  0.8440953465230294


### Ergebnisse interpretieren

- Von Interesse ist der RMSE, weil dieser die gleiche Einheit wie die Zielvariable hat
- RMSE von 0,84 bedeutet, dass es eine Abweichung/Streuung von 84.000€ gibt
- Grundsätzlich ist es ein hoher Fehler, aber es werden nur 3 Prädiktor Variablen verwendet

## Aufgabe 3: Qualitative Variablen California Housing

### Qualitative Variablen California Housing

In [12]:
import pandas as pd
import statsmodels.api as sm

# Quantile AveOccup berechnen, und in Series speichern
groups = pd.qcut(df["AveOccup"], q = 3, labels=['LowAveOccup', 'MediumAveOccup', 'HighAveOccup'])

# Dummies aus Series erstellen, in groups wird erste Kategorie als Baseline-Kategorie genommen (hier dient Low als Baseline)
# Der Datentyp darf nicht default (bool) sein
dummies = pd.get_dummies(groups, None, drop_first=True, dtype=float)

# California Housing und Daummies verknüpfen
df_dummies = pd.concat([df, dummies], axis=1)

features = ["AveOccup", "AveRooms", "MedInc", "MediumAveOccup", "HighAveOccup"]
x_dummies = df_dummies[features]
# intercept hinzufügen
x_dummies = sm.add_constant(x_dummies)

y_dummies = df_dummies["MedHouseVal"]

model = sm.OLS(y_dummies, x_dummies)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            MedHouseVal   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     4920.
Date:                Thu, 27 Nov 2025   Prob (F-statistic):               0.00
Time:                        13:52:03   Log-Likelihood:                -24141.
No. Observations:               20640   AIC:                         4.829e+04
Df Residuals:                   20634   BIC:                         4.834e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              0.9847      0.017     58.

- Theorie wird bestätigt, dass es sich um teure Apartements handelt in denen wenige Menschen drin sind

## Aufgabe 4: Feature Engineering California Housing

### Standardisierung und Skalierung

In [13]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_values = scaler.fit_transform(df[["MedInc", "AveRooms", "AveOccup"]])
df_scaled = pd.DataFrame(scaled_values, columns=["MedInc", "AveRooms", "AveOccup"])
# intercept hinzufügen
df_scaled = sm.add_constant(df_scaled)

y = df_dummies["MedHouseVal"]

model = sm.OLS(y, df_scaled)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            MedHouseVal   R-squared:                       0.481
Model:                            OLS   Adj. R-squared:                  0.481
Method:                 Least Squares   F-statistic:                     6370.
Date:                Thu, 27 Nov 2025   Prob (F-statistic):               0.00
Time:                        13:52:03   Log-Likelihood:                -25477.
No. Observations:               20640   AIC:                         5.096e+04
Df Residuals:                   20636   BIC:                         5.099e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.0686      0.006    357.391      0.0

Vergleich der Koeffizienten:
- Standardisierte Variablen:
    - const          2.0686
    - MedInc         0.8258
    - AveRooms      -0.0948
    - AveOccup      -0.0433 
- Unstandardisierte Variablen:
    - const          0.6069
    - MedInc         0.4347
    - AveRooms      -0.0383
    - AveOccup      -0.0042 

### Erstellen neuer Features:

In [14]:
df_new_feature = df.copy()

df_new_feature["RoomIncomeRatio"] = df_new_feature['MedInc'] / df_new_feature['AveRooms']

x_new_feature = df_new_feature[["MedInc", "AveRooms", "AveOccup", "RoomIncomeRatio"]]

# intercept hinzufügen
x_new_feature = sm.add_constant(x_new_feature)

y_new_feature = df_new_feature["MedHouseVal"]


model = sm.OLS(y_new_feature, x_new_feature)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            MedHouseVal   R-squared:                       0.508
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     5316.
Date:                Thu, 27 Nov 2025   Prob (F-statistic):               0.00
Time:                        13:52:03   Log-Likelihood:                -24932.
No. Observations:               20640   AIC:                         4.987e+04
Df Residuals:                   20635   BIC:                         4.991e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0248      0.025     

Analyse der Auswirkungen:
- P Wert ist überall, außer bei dem const Koeffizienten (Beta0), kleiner als 1%
- F-Statistic Größer als 1, jedoch kleiner als das Modell ohne zusätzlichem Feature
- R-squared: Wir können 50% der Gesamtvarianz erklären (mehr als bei dem Modell ohne zusätzlichem Feature)
- Koeffizient von const und AveOccup ist negativ: 
    - B0 ist negativ, da jedoch der p-Wert größer als 1% kann der Koeffizient vernachlässigt werden?
    - Häuser mit vielen Haushaltsmitliedern sind vielleicht günstiger wie Häuser mit wenigen. 
- Adj. R-squared ist fast genauso groß wie R-squared, das modell wurde nicht durch zu viele Prädiktoren aufgebläht

In [16]:
import statsmodels.api as sm
import pandas as pd

df_interaktionseffekte = df.copy()
df_interaktionseffekte["MedInc_AveRooms"] = df['MedInc'] * df["AveRooms"]
y = df_interaktionseffekte["MedHouseVal"]
features = ["AveOccup", "AveRooms", "MedInc", "MedInc_AveRooms"]
x = df_interaktionseffekte[features]


# intercept hinzufügen
x = sm.add_constant(x)

model = sm.OLS(y, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            MedHouseVal   R-squared:                       0.483
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                     4810.
Date:                Thu, 27 Nov 2025   Prob (F-statistic):               0.00
Time:                        13:52:24   Log-Likelihood:                -25443.
No. Observations:               20640   AIC:                         5.090e+04
Df Residuals:                   20635   BIC:                         5.094e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.3728      0.033     